In [130]:
import os
import numpy as np
import pandas as pd
import datetime as dt
import itertools
import math

In [131]:
#Change the base path before executing code
Base_path = "/home/tennistetris/Documents/Data Analytics/Hall/Inter_hall"
data1 = "extracted_AUM_Data"
data2 = os.path.join("NAV_quarterly_padded")

In [132]:
#Reading the AUM data
Path_to_data = os.path.join(Base_path,data1)
os.chdir(Path_to_data)
db_aum = pd.read_csv('aum_padded.csv')
db_aum.columns = ['Quarter','Data','ID']

In [133]:
#Reading the NAV data
Path_to_data = os.path.join(Base_path , data2)
os.chdir(Path_to_data)
db_nav = pd.read_csv('Combined_Database.csv')
db_nav.drop(columns = ['Unnamed: 0'],inplace = True)
db_nav.columns = ['Quarter','Data','ID']

In [134]:
#Finding a database to define mapping IDs to  - Fund_Class,Fund_Family,Type
os.chdir(Base_path)
db_dividend = pd.read_csv('MF_IIT_DATA_V1_EDITED.csv')
db_dividend['Type'] = 'Dividend'
db_growth = pd.read_csv('MF_IIT_DATA_V3_EDITED.csv')
db_growth['Type'] = 'Growth'
temp = db_dividend[['Id','Fund_Family','Fund_Class','Type']]
temp = [temp , db_growth[['ID','Fund_Family','Fund_Class','Type']]]
mapping = pd.concat(temp)
print mapping

                      Fund_Class                         Fund_Family  \
0   Sector - Pharma & Healthcare                Reliance Mutual Fund   
1   Sector - Pharma & Healthcare                Reliance Mutual Fund   
2   Sector - Pharma & Healthcare                     SBI Mutual Fund   
3   Sector - Pharma & Healthcare                     SBI Mutual Fund   
4   Sector - Pharma & Healthcare                   Tata Mutual Fund    
5   Sector - Pharma & Healthcare                    UTI Mutual Fund    
6            Sector - Technology  Aditya Birla Sun Life Mutual Fund    
7            Sector - Technology        ICICI Prudential Mutual Fund   
8            Sector - Technology                     SBI Mutual Fund   
9            Sector - Technology                     SBI Mutual Fund   
10           Sector - Technology                   Tata Mutual Fund    
11           Sector - Technology                   Tata Mutual Fund    
12    Sector - Banking & Finance                   Tata Mutual F

In [135]:
#Combining IDs and eliminating NaNs
mapping['Id_new'] = 0 
for i in range(mapping.shape[0]):
        mapping['Id_new'][np.isnan(mapping['ID'])] = mapping['Id'][np.isnan(mapping['ID'])]
        mapping['Id_new'][np.isnan(mapping['Id'])] = mapping['ID'][np.isnan(mapping['Id'])]
mapping.drop(labels = ['Id','ID'],axis = 1,inplace = True)
print mapping

/home/tennistetris/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/tennistetris/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


                      Fund_Class                         Fund_Family  \
0   Sector - Pharma & Healthcare                Reliance Mutual Fund   
1   Sector - Pharma & Healthcare                Reliance Mutual Fund   
2   Sector - Pharma & Healthcare                     SBI Mutual Fund   
3   Sector - Pharma & Healthcare                     SBI Mutual Fund   
4   Sector - Pharma & Healthcare                   Tata Mutual Fund    
5   Sector - Pharma & Healthcare                    UTI Mutual Fund    
6            Sector - Technology  Aditya Birla Sun Life Mutual Fund    
7            Sector - Technology        ICICI Prudential Mutual Fund   
8            Sector - Technology                     SBI Mutual Fund   
9            Sector - Technology                     SBI Mutual Fund   
10           Sector - Technology                   Tata Mutual Fund    
11           Sector - Technology                   Tata Mutual Fund    
12    Sector - Banking & Finance                   Tata Mutual F

In [136]:
#Function for finding the quarters over which we have to loop
List1 = ['01' , '04' , '07' , '10']
List2 = ['2013' , '2014' , '2015' , '2016' , '2017']
Quarters = list(itertools.product(List1,List2))

List3 = list()

for i in range(len(Quarters)):
    temp = Quarters[i]
    temp = temp[1]+'-'+temp[0]
    temp = pd.Period(temp,'Q')
    List3.append(temp)
    
Quarters = pd.Series(List3).sort_values().reset_index(drop = True)

In [137]:
Write_dir = 'Aggregate_NAV_returns_data_quarterly'
Write_path = os.path.join(Base_path,Write_dir)

### Aggregating the NAV_returns

In [142]:
# Aggregating the NAV_Returns
Type = mapping['Type'].unique()
Fund_Class = mapping['Fund_Class'].unique()
Fund_Family = mapping['Fund_Family'].unique()
for Tp in Type:
    for Class in Fund_Class:
        final_db = pd.DataFrame()
        if not (Class+'.csv') in os.listdir(Write_path):
            for Fund_House in Fund_Family:
                cond1 = np.array(mapping['Type'] == Tp)
                cond2 = np.array(mapping['Fund_Class'] == Class)
                cond3 = np.array(mapping['Fund_Family']== Fund_House)
                cond = np.all([cond1,cond2,cond3],axis = 0)
                mapping_subset = mapping[cond]
                IDs = list(mapping_subset['Id_new'].astype(int))
                if len(IDs):
                    print ('IDs :',IDs)
                    db_aum_subset = db_aum[db_aum['ID'].isin(IDs)]
                    db_nav_subset = db_nav[db_nav['ID'].isin(IDs)]
                    #print ('aum table :',db_aum_subset.head)
                    #print ('nav table :',db_nav_subset.head)
                    for Quarter in Quarters:
                        #print type(Quarter)
                        Index1 = np.zeros(db_aum_subset.shape[0],dtype = 'bool') # This logical array is used to index the aum data
                        Index2 = np.zeros(db_nav_subset.shape[0],dtype = 'bool') # This logical array is used to index the nav data
                        db_aum_subset_quarter = db_aum_subset['Quarter']
                        db_nav_subset_quarter = db_nav_subset['Quarter']
                        for i in range(len(Index1)):
                            if db_aum_subset_quarter.iloc[i] == Quarter:
                                Index1[i] = 1
                        for i in range(len(Index2)):
                            if db_nav_subset_quarter.iloc[i] == Quarter:
                                Index2[i] = 1
                        db_aum_subset['Data'][Index1]
                        db_nav_subset['Data'][Index2]
                        #print ('aum table :',db_aum_subset.head)
                        #print ('nav table :',db_nav_subset.head)
                        #Now to divide the datasets by ID
                        grouped_aum = db_aum_subset.groupby('ID')
                        key_grouped_aum = grouped_aum.groups.keys()
                        grouped_nav = db_nav_subset.groupby('ID')
                        key_grouped_nav = grouped_nav.groups.keys()
                        if len(set(key_grouped_aum) & set(key_grouped_nav)) == len(key_grouped_aum):
                            for key_aum,group_aum in grouped_aum:
                                #print ('Key_aum: ',key_aum)
                                #print (group_aum)
                                num = 0
                                denum = 0
                                for key_nav,group_nav in grouped_nav:
                                    if key_nav == key_aum:
                                        #print ('Key_nav : ',key_nav)
                                        data1 = np.array(group_nav['Data'])
                                        #print ('data 1 : ',data1)
                                        data2 = np.array(group_aum['Data'])
                                        #print ('data 2 :',data2)
                                        num = num + (data1*data2)
                                        denum = denum + (data2)
                            mean = num/denum
                            #print ('mean : ',mean)
                        os.chdir(Write_path)
                        final_db[Fund_House] = mean
                final_db.to_csv(Class+'.csv')                

In [ ]:
#Loading ths Sharpe Ratio Data

### Aggregating the Sharpe Ratio

In [71]:
# Aggregating the Sharpe Returns
Type = mapping['Type'].unique()
Fund_Class = mapping['Fund_Class'].unique()
Fund_Family = mapping['Fund_Family'].unique()
for Tp in Type:
    for Class in Fund_Class:
        final_db = pd.DataFrame()
        if not (Class+'.csv') in os.listdir(Write_path):
            for Fund_House in Fund_Family:
                cond1 = np.array(mapping['Type'] == Tp)
                cond2 = np.array(mapping['Fund_Class'] == Class)
                cond3 = np.array(mapping['Fund_Family']== Fund_House)
                cond = np.all([cond1,cond2,cond3],axis = 0)
                mapping_subset = mapping[cond]
                IDs = list(mapping_subset['Id_new'].astype(int))
                if len(IDs):
                    print ('IDs :',IDs)
                    db_aum_subset = db_aum[db_aum['ID'].isin(IDs)]
                    db_nav_subset = db_nav[db_nav['ID'].isin(IDs)]
                    #print ('aum table :',db_aum_subset.head)
                    #print ('nav table :',db_nav_subset.head)
                    for Quarter in Quarters:
                        #print type(Quarter)
                        Index1 = np.zeros(db_aum_subset.shape[0],dtype = 'bool') # This logical array is used to index the aum data
                        Index2 = np.zeros(db_nav_subset.shape[0],dtype = 'bool') # This logical array is used to index the nav data
                        db_aum_subset_quarter = db_aum_subset['Quarter']
                        db_nav_subset_quarter = db_nav_subset['Quarter']
                        for i in range(len(Index1)):
                            if db_aum_subset_quarter.iloc[i] == Quarter:
                                Index1[i] = 1
                        for i in range(len(Index2)):
                            if db_nav_subset_quarter.iloc[i] == Quarter:
                                Index2[i] = 1
                        db_aum_subset['Data'][Index1]
                        db_nav_subset['Data'][Index2]
                        #print ('aum table :',db_aum_subset.head)
                        #print ('nav table :',db_nav_subset.head)
                        #Now to divide the datasets by ID
                        grouped_aum = db_aum_subset.groupby('ID')
                        key_grouped_aum = grouped_aum.groups.keys()
                        grouped_nav = db_nav_subset.groupby('ID')
                        key_grouped_nav = grouped_nav.groups.keys()
                        if len(set(key_grouped_aum) & set(key_grouped_nav)) == len(key_grouped_aum):
                            for key_aum,group_aum in grouped_aum:
                                #print ('Key_aum: ',key_aum)
                                #print (group_aum)
                                num = 0
                                denum = 0
                                for key_nav,group_nav in grouped_nav:
                                    if key_nav == key_aum:
                                        #print ('Key_nav : ',key_nav)
                                        data1 = np.array(group_nav['Data'])
                                        #print ('data 1 : ',data1)
                                        data2 = np.array(group_aum['Data'])
                                        #print ('data 2 :',data2)
                                        num = num + (data1*data2)
                                        denum = denum + (data2)
                            mean = num/denum
                            #print ('mean : ',mean)
                        os.chdir(Write_path)
                        final_db[Fund_House] = mean
                final_db.to_csv(Class+'.csv')   

  Quarter   Data      ID
0  2013Q1   6.08  119433
1  2013Q2  14.32  119433
2  2013Q3  21.13  119433
3  2013Q4  29.63  119433
4  2014Q1  40.53  119433
Index([u'Quarter', u'Data', u'ID'], dtype='object')


In [70]:
print (db_nav.head())
print db_nav.columns

  Quarter  Data      ID
0  2013Q1   0.0  135793
1  2013Q2   0.0  135793
2  2013Q3   0.0  135793
3  2013Q4   0.0  135793
4  2014Q1   0.0  135793
Index([u'Quarter', u'Data', u'ID'], dtype='object')


In [83]:
print key_grouped_aum

[100352, 120587]


In [91]:
len(set(key_grouped_aum) & set(key_grouped_nav))

2

In [125]:
print str(Fund_Class[0])+'.csv'

Sector - Pharma & Healthcare.csv
